In [63]:
from keras.layers import *
from keras import Model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr
import keras.losses as losses
 
 
# Setup the network parameters:
intermediate_dim = 300
latent_dim = 10
beta = 5
n_sequences = 200
n_features = 8

boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
# boat_csv = boat_csv[:5739]
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_curved = pd.read_csv("Data/Boat_anomalous_big.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved)
print(anomalous_data.shape)


def prepare_sequences(data, batch_size, interval):
    samples = []
    for i in range(0,data.shape[0]- batch_size, interval):
        sample = data[i:i+batch_size]	
        samples.append(sample)

    sequences = np.array(samples)

    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))

    return trainX

trainX_nominal = prepare_sequences(normal_data,656,656) 
print(trainX_nominal.shape)

input_length = trainX_nominal.shape[0]
trainX_anomalous = prepare_sequences(anomalous_data,6600,6600)
print(trainX_anomalous.shape)  


(66256, 8)


(72600, 8)
(100, 656, 8)
(10, 6600, 8)


In [ ]:

def create_model(pool_size,up_size, windows, seq_length, trainX, epochs):
    encoder = Sequential(name="encoder")
    encoder.add(Conv1D(filters=300, kernel_size=n_features,
               activation='relu',padding='same', name='Conv1'))
    encoder.add(MaxPooling1D(pool_size=pool_size[0], name='Maxpool1'))
    encoder.add(Conv1D(filters=200, kernel_size=n_features,
               activation='relu',padding='same', name='Conv2'))
    encoder.add(MaxPooling1D(pool_size=pool_size[1],  name="Maxpool2"))
    encoder.add(Conv1D(filters=150, kernel_size=n_features,
               activation='relu',padding='same',name='Conv3'))
    encoder.add(MaxPool1D(pool_size=pool_size[2], name="Maxpool3"))
    encoder.add(Conv1D(filters=100, kernel_size=n_features,
               activation='relu',padding="same", name='Conv4'))
    encoder.add(MaxPool1D(pool_size=pool_size[3], name="Maxpool4"))
    encoder.add(Conv1D(filters=latent_dim, kernel_size=n_features,
               activation='relu',
               padding="same", name='Conv5'))
    encoder.add(MaxPool1D(pool_size=pool_size[4], name="Maxpool5"))
    encoder.add(Flatten())
    encoder.add(Dense(latent_dim, name='embedding'))
    
    decoder = Sequential(name="decoder")
    decoder.add(Dense(seq_length))
    decoder.add(Reshape((windows, np.prod(up_size))))
    decoder.add(Conv1D(filters=n_features, kernel_size=n_features,
               activation='relu',padding='same', name='conv-decode1'))
    decoder.add(UpSampling1D(size=up_size[0], name='upsampling1'))
    decoder.add(Conv1D(filters=150, kernel_size=n_features,
               activation='relu',padding='same', name='conv-decode2'))
    decoder.add(UpSampling1D(size=up_size[1], name='upsampling2'))
    decoder.add(Conv1D(filters=200, kernel_size=n_features,
               activation='relu',padding='same', name='conv-decode3'))
    decoder.add(UpSampling1D(size=up_size[2], name='upsampling3'))
    decoder.add(Conv1D(filters=n_features, kernel_size=n_features,
               padding='same', name='conv-decode4'))
    decoder.add(UpSampling1D(size=up_size[3], name='upsampling4'))      
    
    model = Sequential()
    model.add(encoder)
    model.add(decoder)
    
    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])
    # model.summary()

    model.fit(x=trainX, y=trainX, epochs=epochs)
    model.save("Models/Anom_conv_compressor.h5")
    
    return model
    
    
# model = create_model(pool_size=[2, 2, 2, 2, 2], windows=41,
#                      up_size=[2, 2, 2, 2], seq_length=656, 
#                      trainX=trainX_nominal, epochs=15)
model = create_model(pool_size=[2, 2, 2, 5, 5], windows=33, 
                     up_size=[5, 5, 4, 2], seq_length=6600, 
                     trainX=trainX_anomalous, epochs=50)


Epoch 1/15


 32/100 [========>.....................] - ETA: 16s - loss: 1.0875 - acc: 0.1046

 64/100 [==================>...........] - ETA: 5s - loss: 1.7355 - acc: 0.2276 

 96/100 [===========================>..] - ETA: 0s - loss: 1.3358 - acc: 0.3214

100/100 [==============================] - 11s 106ms/step - loss: 1.3090 - acc: 0.3223


...

10/10 [==============================] - 4s 391ms/step - loss: 0.0083 - acc: 0.9091


Epoch 49/50


10/10 [==============================] - 4s 401ms/step - loss: 0.0080 - acc: 0.9106


Epoch 50/50


10/10 [==============================] - 4s 391ms/step - loss: 0.0078 - acc: 0.9128


In [64]:
def create_conv_model(epochs, seq_length, trainX):
    model = Sequential()
    
    encoder = Sequential()
    encoder.add(Conv1D(filters=200, input_shape=(seq_length,8), padding='same', 
                       kernel_size=n_features))
    encoder.add(Conv1D(filters=150, padding='same', 
                       kernel_size=n_features))
    encoder.add(Conv1D(filters=100,padding='same', 
                       kernel_size=n_features))
    encoder.add(Conv1D(filters=50, padding='same',
                       kernel_size=n_features))
    encoder.add(Conv1D(filters=latent_dim, padding='same',
                       kernel_size=n_features)) 
    
    decoder = Sequential(name="Decoder")
    decoder.add(Dense(64, name="Decoder_first"))
    decoder.add(Dense(units=n_features))
    
    model.add(encoder)
    model.add(decoder)
    
    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])
    
    model.fit(x=trainX, y=trainX, epochs=epochs)
    #model.save("Models/Nominal_conv.h5")

    return model

nom_model = create_conv_model(15, 656, trainX_nominal)

anom_model = create_conv_model(50, 6600, trainX_anomalous)


Epoch 1/15


 32/100 [========>.....................] - ETA: 16s - loss: 1.0875 - acc: 0.1046

 64/100 [==================>...........] - ETA: 5s - loss: 1.7355 - acc: 0.2276 

 96/100 [===========================>..] - ETA: 0s - loss: 1.3358 - acc: 0.3214

100/100 [==============================] - 11s 106ms/step - loss: 1.3090 - acc: 0.3223


...

10/10 [==============================] - 4s 391ms/step - loss: 0.0083 - acc: 0.9091


Epoch 49/50


10/10 [==============================] - 4s 401ms/step - loss: 0.0080 - acc: 0.9106


Epoch 50/50


10/10 [==============================] - 4s 391ms/step - loss: 0.0078 - acc: 0.9128


In [52]:
nom_model = kr.models.load_model("Models/Nominal_conv_compressor.h5")
anom_model = kr.models.load_model("Models/Anom_conv_compressor.h5") 

print("Done")


(100, 656, 8)


(10, 6600, 8)


In [67]:

nom_encoded = nom_model.get_layer(index=0).predict(trainX_nominal)
print(nom_encoded.shape)
# 
# plt.plot(nom_encoded[0])
# plt.show()

anom_encoded = anom_model.get_layer(index=0).predict(trainX_anomalous) 
# 
# plt.plot(anom_encoded)
# plt.show()


(100, 656, 8)


(10, 6600, 8)


In [69]:

nom_dec = nom_model.get_layer(index=1).predict(nom_encoded)
print(nom_dec.shape)

anom_dec = anom_model.get_layer(index=1).predict(anom_encoded)
print(anom_dec.shape)

(100, 656, 8)


(10, 6600, 8)


In [70]:

from sklearn.manifold import TSNE

def tsne(data, title):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(data)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"],y=tsne_df["Y"], alpha=0.5)
    plt.title(title)
    plt.show()
    
    return tsne_df


In [71]:
weight_modifiers = np.ones((latent_dim,))

tsne_dec_nom_df = tsne(nom_dec[0], "Decoded Nominal")
print(len(tsne_dec_nom_df))

tsne_dec_anom_df = tsne(anom_dec[0], "Decoded Anomalous")
print(len(tsne_dec_anom_df))


656


6600


In [72]:
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

plt.figure(figsize=(10, 7))
plt.title("Customer Dendograms")
dend_nom = shc.dendrogram(shc.linkage(tsne_dec_nom_df, method='ward'))
dend_anom = shc.dendrogram(shc.linkage(tsne_dec_anom_df, method='ward'))



In [74]:

n_clusters = 4
cluster = AgglomerativeClustering(n_clusters=n_clusters, 
                                  affinity='euclidean',
                                  linkage='ward')
cl_nom = cluster.fit_predict(tsne_dec_nom_df)
cl_anom = cluster.fit_predict(tsne_dec_anom_df)
plt.plot(cl_nom)
plt.title("NOMINAL CLUSTERS")
plt.show()

plt.plot(cl_anom)
plt.title("Anomalous_clusters")
plt.show()

In [75]:

df_nom = pd.DataFrame(normal_data, columns=boat_csv.columns)
df_anom = pd.DataFrame(anomalous_data, columns=boat_curved.columns)
def plot_clusters(cl, df, nominal):
    for i in range(n_clusters):
        cluster = np.squeeze(np.argwhere(cl==i))    
        if nominal:
            plt.scatter(x=df['Lon'][cluster],y=df["Lat"][cluster],s=5)
        else:
            plt.scatter(x=df['lon'][cluster],y=df["lat"][cluster],s=5)
    plt.show()


plot_clusters(cl_nom, df_nom, True)
plot_clusters(cl_anom, df_anom, False)


In [32]:
df_nominal = pd.DataFrame(normal_data, columns=boat_csv.columns)
df = pd.DataFrame(nom_enc[0])

plt.figure(1)
axis_list = []

for i in range(df.shape[1]):
    plt.plot(df[i])

plt.show()    

In [39]:
from sklearn.cluster import KMeans
cluster_comp = [0,1,2,3]# print(tsne_enc_df)
# print(tsne_dec_df)

kmeans_normal = KMeans(n_clusters=2, random_state=0).fit(normal_data)
clusters_normal = kmeans_normal.predict(np.average(trainX_nominal,axis=0))

kmeans_enc = KMeans(n_clusters=2, random_state=0).fit(tsne_enc_df)
clusters_enc = kmeans_enc.predict(tsne_enc_df)
plt.plot(clusters_enc)
plt.show()

kmeans_dec = KMeans(n_clusters=2, random_state=0).fit(tsne_dec_df)
clusters_dec = kmeans_dec.predict(tsne_dec_df)

In [40]:
def transform_to_mask(cl_label, clust_obj):
    mask = []
    part = []
    for i, elem in enumerate(clust_obj):
        if elem == cl_label:
            part.append(i)
        else:
            if part:
                mask.append(part)
                part = []
            else:
                pass
    return mask


def get_mask_list(clust_obj):
    mask_list = []
    for i in cluster_comp:
        mask_list.append(transform_to_mask(i, clust_obj))
    return mask_list


masks_normal = np.array(get_mask_list(clusters_normal))

masks_enc = np.array(get_mask_list(clusters_enc))

masks_dec = np.array(get_mask_list(clusters_dec))

masks = (masks_normal, masks_enc,masks_dec)
print(masks_normal.shape, masks_enc.shape, masks_enc.shape)


(4,) (4,) (4,)


In [41]:
def plot_cl(cl, color):
    plt.plot(cl["lon"], cl["lat"], color=color)
        
    
def plot_clusters_on_map():
    titles = ['Nominal Normal','Nominal Encoded','Nominal Decoded']
    color_list = ['blue','green','red','black']
    map = anomalous_data[:6620]
    print(map.shape)
    for k,mask in enumerate(masks):
        for i, elem in enumerate(mask):
            for j in elem:
                cl = pd.DataFrame(map[j], columns=boat_curved.columns)
                plot_cl(cl, color_list[i]) 
        plt.title(titles[k])
        plt.show()
      
               
plot_clusters_on_map()


(6620, 7)
